In [1]:
pip install yfinance plotly pandas numpy


In [2]:
# ============================================================
# HINDALCO – CORPORATE FINANCE DASHBOARD (FINAL ++ PRO VERSION)
# ============================================================

import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import base64, os
from plotly.offline import plot

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
STOCK = "HINDALCO.NS"
MARKET = "^NSEI"
START = "2022-04-01"
END = "2025-03-31"

RISK_FREE = 0.06
ERP = 0.07
TAX = 0.2836

PROJECT_IRR = 0.135
PROJECT_BETA = 0.74

# ------------------------------------------------------------
# 1. MARKET DATA
# ------------------------------------------------------------
prices = yf.download([STOCK, MARKET], start=START, end=END)["Close"].dropna()
prices.columns = ["Hindalco", "Nifty"]

returns = np.log(prices / prices.shift(1)).dropna()

# ------------------------------------------------------------
# 2. BETA & RISK
# ------------------------------------------------------------
beta = np.cov(returns["Hindalco"], returns["Nifty"])[0,1] / np.var(returns["Nifty"])
rolling_beta = returns["Hindalco"].rolling(60).cov(returns["Nifty"]) / returns["Nifty"].rolling(60).var()
rolling_vol = returns["Hindalco"].rolling(30).std() * np.sqrt(252)
drawdown = prices["Hindalco"] / prices["Hindalco"].cummax() - 1
sharpe = (returns["Hindalco"].rolling(30).mean()*252 - RISK_FREE) / rolling_vol

# ------------------------------------------------------------
# 3. COST OF CAPITAL
# ------------------------------------------------------------
market_cap = 202464
debt = 63644

ke = RISK_FREE + beta * ERP
kd = 0.075 * (1 - TAX)

we = market_cap / (market_cap + debt)
wd = debt / (market_cap + debt)
wacc = we*ke + wd*kd

# ------------------------------------------------------------
# 4. CORE VISUALS
# ------------------------------------------------------------
norm = prices / prices.iloc[0] * 100
cum_returns = (1 + returns).cumprod() * 100

fig_price = px.line(norm, title="Normalized Price Performance (Base = 100)",
                     template="plotly_dark", height=260)

fig_cum = px.line(cum_returns, title="Cumulative Returns Comparison",
                   template="plotly_dark", height=260)

fig_returns = px.histogram(returns, x="Hindalco", nbins=60,
                            title="Daily Return Distribution",
                            template="plotly_dark", height=260)

fig_neg = px.histogram(returns[returns["Hindalco"]<0], x="Hindalco",
                       title="Downside Risk Distribution (Losses Only)",
                       template="plotly_dark", height=260)

fig_beta = px.bar(x=["Firm Beta","Project Beta"], y=[beta,PROJECT_BETA],
                  title="Firm vs Project Beta",
                  template="plotly_dark", text_auto=".2f", height=220)

fig_roll_beta = px.line(rolling_beta, title="Rolling 60-Day Beta",
                         template="plotly_dark", height=240)

fig_vol = px.line(rolling_vol, title="Rolling Volatility (30D)",
                   template="plotly_dark", height=240)

fig_sharpe = px.line(sharpe, title="Rolling Sharpe Ratio (30D)",
                      template="plotly_dark", height=240)

fig_drawdown = px.line(drawdown, title="Drawdown Analysis",
                        template="plotly_dark", height=240)

# ------------------------------------------------------------
# 5. RISK–RETURN MAP
# ------------------------------------------------------------
fig_rr = px.scatter(
    x=[rolling_vol.mean(), returns["Nifty"].std()*np.sqrt(252)],
    y=[returns["Hindalco"].mean()*252, returns["Nifty"].mean()*252],
    size=[beta,1],
    text=["Hindalco","Market"],
    labels={"x":"Volatility","y":"Expected Return"},
    title="Risk–Return Profile",
    template="plotly_dark",
    height=260
)

# ------------------------------------------------------------
# 6. CAPM – SECURITY MARKET LINE
# ------------------------------------------------------------
beta_range = np.linspace(0,1.5,25)
sml = RISK_FREE + beta_range*ERP

fig_sml = go.Figure()
fig_sml.add_trace(go.Scatter(x=beta_range,y=sml,mode="lines",name="SML"))
fig_sml.add_trace(go.Scatter(x=[beta],y=[ke],mode="markers",
                             marker=dict(size=10),name="Hindalco"))
fig_sml.update_layout(title="CAPM – Security Market Line",
                      template="plotly_dark", height=260)

# ------------------------------------------------------------
# 7. CAPITAL STRUCTURE
# ------------------------------------------------------------
fig_cap = px.pie(names=["Equity","Debt"], values=[we,wd],
                 hole=0.4, title="Capital Structure",
                 template="plotly_dark", height=260)

# ------------------------------------------------------------
# 8. HEADER DATA
# ------------------------------------------------------------
latest_price = prices.iloc[-1]["Hindalco"]
daily_ret = returns.iloc[-1]["Hindalco"]*100

# ------------------------------------------------------------
# 9. STRATEGIC NEWS (TOP BANNER)
# ------------------------------------------------------------
news_html = """
<p><b>Hindalco accelerates aluminium recycling capacity</b><br>
<span style='color:#9ca3af'>Circular economy & ESG-led margin stability</span></p>

<p><b>Novelis expands downstream integration</b><br>
<span style='color:#9ca3af'>Reduces cyclicality, stabilises beta</span></p>

<p><b>AA+ credit rating reaffirmed</b><br>
<span style='color:#9ca3af'>Supports low cost of debt & capital flexibility</span></p>
"""

# ------------------------------------------------------------
# 10. LOGO
# ------------------------------------------------------------
logo_html = "<h2 style='color:#38bdf8'>HINDALCO</h2>"
if os.path.exists("hindalco_logo.png"):
    with open("hindalco_logo.png","rb") as f:
        logo_html = f"<img src='data:image/png;base64,{base64.b64encode(f.read()).decode()}' height='45'>"

# ============================================================
# 11. OPTIMIZED EXPORT (SIZE-SAFE FOR GITHUB)
# ============================================================

from plotly.io import to_html

def compact(fig):
    fig.update_layout(
        hovermode="x unified",
        showlegend=True,
        margin=dict(l=30, r=30, t=40, b=30)
    )
    fig.update_traces(
        hoverinfo="skip",
        marker_line_width=0
    )
    return fig

figs = [
    fig_price, fig_cum, fig_rr, fig_returns, fig_neg,
    fig_beta, fig_roll_beta, fig_vol,
    fig_sharpe, fig_drawdown, fig_sml, fig_cap
]

html_blocks = [
    to_html(compact(fig),
            include_plotlyjs=False,
            full_html=False,
            config={"displayModeBar": False})
    for fig in figs
]

with open("hindalco_dashboard.html", "w", encoding="utf-8") as f:
    f.write(f"""
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>Hindalco – Corporate Finance Dashboard</title>

<!-- Load Plotly ONCE -->
<script src="https://cdn.plot.ly/plotly-2.27.0.min.js"></script>

<style>
body {{
    margin:0;
    font-family:Inter,Arial;
    background:#0b1220;
    color:#e5e7eb;
}}
.header {{
    position:fixed;
    top:0; left:0; right:0;
    height:75px;
    background:#020617;
    padding:15px 25px;
    border-bottom:1px solid #1e293b;
    z-index:10;
}}
.dashboard {{
    margin-top:95px;
    padding:25px;
    display:grid;
    grid-template-columns:repeat(auto-fit,minmax(420px,1fr));
    gap:22px;
}}
.card {{
    background:#020617;
    border-radius:14px;
    padding:18px;
    box-shadow:0 0 0 1px #1e293b;
}}
.news {{
    grid-column:1/-1;
    background:linear-gradient(135deg,#020617,#052e16);
}}
</style>
</head>

<body>

<div class="header">
<h2 style="color:#38bdf8">Hindalco – Corporate Finance Dashboard</h2>
<div style="float:right">
<b>₹{round(latest_price,2)}</b> | {round(daily_ret,2)}%
</div>
</div>

<div class="dashboard">

<div class="card news">
{news_html}
</div>

""")


    for block in html_blocks:
        f.write(f"<div class='card'>{block}</div>\n")

    f.write("""
</div>
</body>
</html>
""")

print("✅ OPTIMIZED DASHBOARD GENERATED (≤25 MB)")


/tmp/ipython-input-1311580489.py:31: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  2 of 2 completed


✅ OPTIMIZED DASHBOARD GENERATED (≤25 MB)


In [3]:
from google.colab import files
files.download("hindalco_dashboard.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%writefile app.py
# ============================================================
# HINDALCO – CORPORATE FINANCE DASHBOARD (STREAMLIT)
# ============================================================

import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# ------------------------------------------------------------
# PAGE CONFIG
# ------------------------------------------------------------
st.set_page_config(
    page_title="Hindalco – Corporate Finance Dashboard",
    layout="wide",
    initial_sidebar_state="expanded"
)

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
STOCK = "HINDALCO.NS"
MARKET = "^NSEI"
START = "2022-04-01"
END = "2025-03-31"

RISK_FREE = 0.06
ERP = 0.07
TAX = 0.2836
PROJECT_BETA = 0.74
PROJECT_IRR = 0.135

# ------------------------------------------------------------
# DATA LOAD (CACHED)
# ------------------------------------------------------------
@st.cache_data
def load_data():
    prices = yf.download([STOCK, MARKET], start=START, end=END)["Close"]
    prices = prices.resample("W").last().dropna()
    prices.columns = ["Hindalco", "Nifty"]
    returns = np.log(prices / prices.shift(1)).dropna()
    return prices, returns

prices, returns = load_data()

# ------------------------------------------------------------
# METRICS
# ------------------------------------------------------------
beta = np.cov(returns["Hindalco"], returns["Nifty"])[0,1] / np.var(returns["Nifty"])
ke = RISK_FREE + beta * ERP

market_cap = 202464
debt = 63644
we = market_cap / (market_cap + debt)
wd = debt / (market_cap + debt)
kd = 0.075 * (1 - TAX)
wacc = we * ke + wd * kd

rolling_beta = returns["Hindalco"].rolling(12).cov(returns["Nifty"]) / returns["Nifty"].rolling(12).var()
rolling_vol = returns["Hindalco"].rolling(8).std() * np.sqrt(52)
drawdown = prices["Hindalco"] / prices["Hindalco"].cummax() - 1

# ------------------------------------------------------------
# HEADER
# ------------------------------------------------------------
st.title("🏭 Hindalco Industries – Corporate Finance Dashboard")
st.markdown("**AI-assisted Risk, CAPM & Capital Structure Analysis**")

st.info("""
• Circular economy transition
• Firm vs project risk separation
• CAPM, WACC & investment decision framework
""")

# ------------------------------------------------------------
# KPI ROW
# ------------------------------------------------------------
c1, c2, c3, c4 = st.columns(4)
c1.metric("Firm Beta", f"{beta:.2f}")
c2.metric("Cost of Equity", f"{ke*100:.2f}%")
c3.metric("WACC", f"{wacc*100:.2f}%")
c4.metric("Project IRR", f"{PROJECT_IRR*100:.1f}%")

# ------------------------------------------------------------
# PRICE PERFORMANCE
# ------------------------------------------------------------
st.subheader("📈 Market Performance")

norm = prices / prices.iloc[0] * 100
fig_price = px.line(
    norm,
    title="Normalized Price Performance (Base = 100)",
    template="plotly_dark"
)
st.plotly_chart(fig_price, use_container_width=True)

# ------------------------------------------------------------
# RISK & BETA
# ------------------------------------------------------------
col1, col2 = st.columns(2)

with col1:
    fig_beta = px.line(
        rolling_beta,
        title="Rolling Beta (12-Week)",
        template="plotly_dark"
    )
    st.plotly_chart(fig_beta, use_container_width=True)

with col2:
    fig_vol = px.line(
        rolling_vol,
        title="Rolling Volatility (Annualised)",
        template="plotly_dark"
    )
    st.plotly_chart(fig_vol, use_container_width=True)

# ------------------------------------------------------------
# CAPM – SECURITY MARKET LINE
# ------------------------------------------------------------
st.subheader("📐 CAPM – Security Market Line")

beta_range = np.linspace(0, 1.5, 30)
sml = RISK_FREE + beta_range * ERP

fig_sml = go.Figure()
fig_sml.add_trace(go.Scatter(x=beta_range, y=sml, mode="lines", name="SML"))
fig_sml.add_trace(go.Scatter(
    x=[beta], y=[ke], mode="markers",
    marker=dict(size=10, color="lime"),
    name="Hindalco"
))

fig_sml.update_layout(template="plotly_dark")
st.plotly_chart(fig_sml, use_container_width=True)

# ------------------------------------------------------------
# DRAWDOWN
# ------------------------------------------------------------
st.subheader("⚠️ Downside Risk")

fig_dd = px.line(
    drawdown,
    title="Drawdown Analysis",
    template="plotly_dark"
)
st.plotly_chart(fig_dd, use_container_width=True)

# ------------------------------------------------------------
# PROJECT VS FIRM
# ------------------------------------------------------------
st.subheader("🏗 Project vs Firm Risk")

fig_pf = px.bar(
    x=["Firm Beta", "Project Beta"],
    y=[beta, PROJECT_BETA],
    text_auto=".2f",
    title="Risk Comparison",
    template="plotly_dark"
)
st.plotly_chart(fig_pf, use_container_width=True)

# ------------------------------------------------------------
# STRATEGIC CONTEXT
# ------------------------------------------------------------
st.subheader("📰 Strategic Context")

st.markdown("""
- **Recycling expansion** reduces energy intensity and beta volatility
- **Downstream integration (Novelis)** stabilizes cash flows
- **AA+ credit rating** supports low cost of debt
""")

st.success("✅ Dashboard ready for academic submission & viva demonstration")


Writing app.py
